# Introduction to Yao's Garbled Circuits
Yao's Garbled Circuits is a cryptographic protocol introduced by Andrew Yao in 1982 that enables secure two-party computation. It allows two parties, let's call them Alice and Bob, to compute a function on their private inputs without revealing their inputs to each other. In this blog post, we'll explore the concept of Yao's Garbled Circuits and provide Python code examples to demonstrate its implementation.

## How it works
Yao's Garbled Circuits is a form of MPC where there are only two parties and each party is assumed to be "honest-but-curious" meaning that they might try and look at things they shouldn't, but they won't actively try and break the protocol. The participants are trying to compute the function $f(x, y)$ which is a boolean circuit meaning that all inputs are 0 or 1 and there are nodes like AND, OR, and NOT gates that connect inputs. The function is known to both parties, but they don't want to reveal their inputs to each other. The protocol works as follows:

1. Alice and Bob agree on a boolean circuit $f(x)$ that they want to compute
2. Alice garbles the circuit and sends it along with her encrypted input $x0$ to Bob
3. Bob encrypts his input $x1$ by using an Oblivious Transfer protocol with Alice
4. Bob evaluates the garbled circuit with his encrypted input and sends the result to Alice

Lets go through each of these steps in more detail.

In this example we are going to compute the function $f(x, y) = x \oplus y$ where $\oplus$ is the XOR operator. Alice has input $x = 1$ and Bob has input $y = 0$. The first step is to agree on a boolean circuit that computes $f(x, y)$.

![](diagram1.png){width="40%"}

Then Alice will generate a key for each wire and each possible value for the wire. In this case there are two wires and each wire can be 0 or 1 so there are four keys. 
![](diagram2.png){width="40%"}

Alice encrypts each output value with the corresponding keys and permutes the table randomly. Then she sends the garbled circuit to Bob along with her encrypted input $x0$.
![](diagram3.png)

Bob uses an [Oblivious Transfer protocol](https://www.trevortomlin.com/posts/ot/) to select which key he wants from alice.
![](diagram4.png)

Since Bob now has Alice's key and the key for his own input, he can decrypt the output value and send it to Alice. Bob either has to go through every entry in the table and try to see if it decrypts, or there can be a select bit specified in the protocol so he knows where to look. Bob then sends the output value to Alice.

## Multi Gate Circuits
For multi gate circuits instead of calculating the encryption directly on the function output, the garbler will also generate a key for the output and encrypt that key instead. Then when Bob decrypts the circuit, he will get the output key which can be fed into other circuits.

## Example Code

In [ ]:
import os
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

# store the xor table in a dictionary
xor_table = {
    (0, 0): 0,
    (0, 1): 1,
    (1, 0): 1,
    (1, 1): 0
}

# generate a random key
def generate_key():
    return os.urandom(16)

# encrypt a message with a key
def encrypt(key, message):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(pad(str(message).encode(), 16))

# decrypt a message with a key
def decrypt(key, message):
    cipher = AES.new(key, AES.MODE_ECB)
    return unpad(cipher.decrypt(str(message).encode()), 16)

# garble the circuit
def garble_circuit():
    # generate keys for each wire and each possible value
    keys = {
        "x0": {
            0: generate_key(),
            1: generate_key()
        },
        "x1": {
            0: generate_key(),
            1: generate_key()
        },
        "f(x0, x1)": {
            0: 0,
            1: 1
        }
    }

    # encrypt each output value with the corresponding keys
    permuted_table = {
        encrypt(keys["x0"][0], encrypt(keys["x1"][0], keys["f(x0, x1)"][xor_table[(0, 0)]])),
        encrypt(keys["x0"][0], encrypt(keys["x1"][1], keys["f(x0, x1)"][xor_table[(0, 1)]])),
        encrypt(keys["x0"][1], encrypt(keys["x1"][0], keys["f(x0, x1)"][xor_table[(1, 0)]])),
        encrypt(keys["x0"][1], encrypt(keys["x1"][1], keys["f(x0, x1)"][xor_table[(1, 1)]]))
    }

    return permuted_table, keys

print("XOR Table")
print("x0\tx1\tf(x0, x1)")
for x0 in range(2):
    for x1 in range(2):
        print(f"{x0}\t{x1}\t{xor_table[(x0, x1)]}")

print("-" * 20)

# Alice garbles the circuit and sends it to Bob
permuted_table, keys = garble_circuit()

print("Garbled Table")
print("x0\tx1")
for x0 in range(2):
    for x1 in range(2):
        print(f"{x0}\t{x1}\t{permuted_table[(x0, x1)]}")

print(keys)

x0 = 0

# Bob evaluates the circuit
x1 = 1

output = evaluate_circuit(permuted_table, x0, x1)

## Applications

## Conclusion